In [1]:
import nltk
from nltk.corpus import words
import re
from pattern.en import suggest
filename = "mlp_001.txt"
field = 'mylittlepony'
filepath = 'data/{field}/{name}'.format(field=field,name=filename)

/Users/bagdongmin/anaconda2/lib/python2.7/site-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


In [2]:

#----------by DM-----------

#input : word, output : word

#---------------------



#...제외 3개이상 알파벳 반복 -> 2개
def three2two(word):
    pattern = re.compile(r'([^\.]+)\1{2,}')
    return pattern.sub(r'\1\1', word)

# word -> word ('helllloooo' -> 'hello')
def correct_lengthen(word):
    short_word = three2two(word)
    if short_word != word:
        p_word = suggest(short_word) #(가장 유사한 단어, 확률) tuples
        short_word = p_word[0][0] #제일 높은 확률의 단어 (limitation : Nnnn -> Anna)
        #print(short_word)
    return short_word

#---------------------




tokens =[]


In [3]:
#------------------

#input : chunk
#output : chunk without speaker 
#ex : "Narrator : Once upon a time,,..." -> "Once upon a time..."
def parse_lv1(chunk):
    newchunk = ''.join(chunk.split(':')[1:])
    return newchunk
#input : chunk without speaker 
#output : chunk without speker/지시문 
#ex : "[seriously] Once upon a time,,..." -> "Once upon a time..."
regex = re.compile(r'\[.+\]')
def parse_lv3(chunk):
    return regex.sub('',chunk)
#input : chunk without speker/지시문 
#output : chuck without speark/지시문 r'-*'
#ex : "H--i" -> "Hi"
def parse_lv4(chunk):
    tokens = []
    for token in chunk.split():
        if '-' in token:
            if token in words.words():# mother-in-law -> should be accepted
                newtoken = [token]
            elif '-' in token: #else case
                newtoken = token.replace('-',' ').split() # basically split the '-' 
                if ''.join(newtoken) in words.words():    # Ex-cuseme -> excusme case
                    
                    newtoken = [''.join(newtoken)]
        else:
            newtoken = [token]
        tokens+=newtoken
    return ' '.join(tokens)
#------------------



In [4]:
#input : string
#output : boolean
#description : check whether string contains alphabet.
def alphabet(s):
    for c in s:
        if (c>='0' and c<='9'):    
            return False
    return True

#input : string
#output : boolen
#description : check whether string's first letter is capital letter.
def checkfirst(s):
    return (s[0]==s[0].upper() and alphabet(s))

#input : list of 덩어리
#output : list of 단어
#description : 겹치는 단어 제거
def remove_prop(lst):
    result=[]
    prop=[]
    for e in lst:
        delimiters = ".", "!", "?", "\n"
        regexPattern = '|'.join(map(re.escape, delimiters))
        tmp = re.split(regexPattern, e)
        tmp = list(filter(None, tmp))
        
        for t in tmp:
            deli = ",", " "
            reg = "|".join(map(re.escape, deli))
            ss=re.split(reg,t.strip())
            ss=list(filter(None, ss))
            #ss=t.strip().split()
            result=result+ss
            S=ss[1:]
            for s in S:
                if(checkfirst(s) and (not s in prop)):
                    prop.append(s)

    result = list(filter(lambda a: not a in prop, result))
    return result


# result=remove_prop(lst)
# print(result)



In [5]:
#input : filepath,filename,field; ex) data/{field}/{filename} -> data/mylittlepony/mlp_001.txt
#output : tokens = [word,word,word,...,word]
def parse(filepath,filename,field):
    #regex = re.compile(r'\[.+\]')
    chunks =[]
    with open(filepath,'r') as f:
        
        for chunk in f: # 덩어리 기준
            chunk = chunk.strip()
            #1 : 지시문, 화자는 제외
            #1-1 화자 제거
            chunk = parse_lv1(chunk)
            #1-2 지시문 제거
            chunk = parse_lv3(chunk)
            #2 -와 -- 제거
            chunk = parse_lv4(chunk)
            #3 lengthenword correcting
            #correct_lengthen(word)
            newchunk = []
            for word in chunk.split():
                newchunk.append(correct_lengthen(word))
            newchunk = ' '.join(newchunk)
            #remove proper noun
            chunks.append(newchunk)
            
        #print(newchunk)
        #4 remove prop
        wordset = remove_prop(chunks)
    return wordset

In [10]:
#tk = parse(filepath,filename,field)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
example_sent = "This is a sample sentence, showing off the stop words filtration."
 
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(example_sent)
 
filtered_sentence = [w for w in word_tokens if not w in stop_words]
 
filtered_sentence = []



In [11]:
data = []
#pony processing


field = 'mylittlepony'
fnumber = 5
for i in range(fnumber):
    filename = "mlp_{idx}.txt".format(idx='{0:03}'.format(i+1))
    filepath = 'data/{field}/{name}'.format(field=field,name=filename)
    data += parse(filepath,filename,field)
    if i%10==0:
        print("doing.. ",i)
        


('doing.. ', 1)
('doing.. ', 2)
('doing.. ', 3)


/Users/bagdongmin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/bagdongmin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:26: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


('doing.. ', 4)


In [12]:
#stopword removing
data = [w for w in data if not w in stop_words]


In [14]:
n = len(data)
n_file = (n//200) +1
offset = n - 200*n_file+200
print(offset, n_file)

for i in range(n_file-1):
    filename = "parsed_mlp_{idx}.txt".format(idx='{0:03}'.format(i+1))
    filepath = 'data/{field}/{name}'.format(field=field,name=filename)
    newdata = data[i*200:(i+1)*200]
    with open(filepath,'w') as f:
        f.write(' '.join(newdata))
        f.write('\n')
filename = "parsed_mlp_{idx}.txt".format(idx='{0:03}'.format(n_file+1))
filepath = 'data/{field}/{name}'.format(field=field,name=filename)
newdata = data[n_file*200:]
with open(filepath,'w') as f:
    with open(filepath,'w') as f:
        f.write(' '.join(newdata))
        f.write('\n')

    

(64, 28)


In [ ]:
#stopwords removing using NLTK I, Mr. / I'll I've  /  
